
**importing modules**
--

In [ ]:
!pip install labelme
!pip uninstall opencv-python-headless==4.5.5.62 
!pip install opencv-python-headless==4.1.2.30
!pip install fiftyone
!pip install torchmetrics
!pip install funcy
!pip install scikit-multilearn

In [1]:
import torch
import os 
import torch.optim as optim
import copy
import numpy as np
import json
import glob
import os.path as osp
import sys
import gc
from torchmetrics.detection.mean_ap import MeanAveragePrecision as MAP
from data import Read_data,detetction_class
from visualize import visualize,post_process

**downloading data**
--

In [ ]:
!git clone https://github.com/2Obe/BSData.git
!git clone https://github.com/akarazniewicz/cocosplit.git

**preparing and reading data**
--

In [4]:
!mkdir coco_folder

In [ ]:
%cd coco_folder
!mkdir data
%cd ..

In [6]:
data=Read_data('BSData/data','BSData/label')
with open('coco_folder/data.json', 'w') as fp:
    json.dump(data, fp,  indent=4)

In [7]:
import shutil
files = glob.iglob(os.path.join("BSData/data", "*.jpg"))
for file in files:
    if os.path.isfile(file):
        shutil.copy2(file, os.path.join("coco_folder", "data"))


**splitting test and train data**
---

In [ ]:
!python cocosplit/cocosplit.py --having-annotations -s 0.8 coco_folder/data.json coco_folder/train.json coco_folder/test.json
print(f"length of train data={len(json.load(open('coco_folder/train.json'))['images'])}")
print(f"length of test data={len(json.load(open('coco_folder/test.json'))['images'])}")

**primary model**
--

In [2]:
from model import get_model
model=get_model(2,'resnset101')


**data class**
--

In [ ]:
train_dataset=detetction_class('/content/coco_folder/train.json','/content/coco_folder/data')
test_dataset=detetction_class('/content/coco_folder/test.json','/content/coco_folder/data')

**training**
--

In [18]:
lr=0.0001#0.0001 
weight_decay=1e-5
for parameter in model.parameters():
  parameter.require_grad=True
for parameter in model.backbone:
  parameter.require_grad= True
optimizer=optim.SGD(model.parameters(),lr=lr,weight_decay=weight_decay,momentum=0.9)
optim_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
#train_sample=[i[0] for i in train_dataset]
#train_label=[i[1] for i in train_dataset]

In [40]:
torch.save(model.state_dict(),'pretrained_rcnn')

In [4]:

model.load_state_dict(torch.load('pretrained_rcnn'))#,map_location='cpu'))

<All keys matched successfully>

In [ ]:
Device='cuda'
epochs=8
batch_size=2
model.to(Device)
model.train(True)
parameter=np.array([6,6,10,10,0.0])#np.array([1,1,1,1,0.5])# #both initial wait are used one for fine tuning and the other is for training
for i in range(epochs):
  if i<8:
    parameter*[0.9,0.9,0.8,0.8,2]#[0.8,0.8,0.8,0.8,1.5]   #both list are used one for fine tuning and the other is for training
  print(f'epoch {i} of {epochs}')
  for i in range(int(len(train_sample)/batch_size)+1):
    optimizer.zero_grad()
    j=(i+1)*batch_size if i<int(len(train_sample)/batch_size) else None

    k=[x.to(Device) for x in train_sample[(i*batch_size):j]]
    l=[{'labels':x['labels'].to(Device),'boxes':x['boxes'].to(Device),'masks':x['masks'].to(Device)
        } for x in train_label[(i*batch_size):j]]
        
    loss=model(k,l)

    del (k)
    del (l)
    torch.cuda.empty_cache()
    
    loss1=loss['loss_classifier']
    loss2=loss['loss_box_reg']
    loss3=loss['loss_mask']
    loss4=loss['loss_objectness']
    loss5=loss['loss_rpn_box_reg']

    total_loss=parameter[0]*loss2+parameter[1]*loss1+parameter[2]*loss4+parameter[3]*loss5+parameter[4]*loss3
    total_loss.backward()
    if(i%20==0):
      print(loss)
      print(total_loss.item())

    optimizer.step()
  optim_scheduler.step()
  

**visualize results**
---

In [ ]:
k=test_dataset[3]

visualize(k[0],k[1],1)

In [ ]:
model.train(False)
Device='cpu'
model.to(Device)
z=[k[0].to(Device)]
f=model(z)
del z
torch.cuda.empty_cache()

In [ ]:
visualize(k[0],f[0],1,0.1)

**testing**
---

In [11]:
def get_target_prediction(model,sample,Device):
  z=[sample[0].to(Device)]
  f=model(z)
  f=post_process(f[0],0.5,0.1)
  k={"boxes":f["boxes"].to("cpu"),"labels":f["labels"].to("cpu"),"scores":f["scores"].to("cpu")}
  del (f)
  del (z)
  gc.collect()
  return  k

In [ ]:
model.train(False)
Device='cuda'
model.to(Device)
j=0
x,y=[],[]
map=MAP()
for i in test_dataset:
  with torch.no_grad():
    x.append(get_target_prediction(model,i,Device))
    y.append({"boxes":i[1]["boxes"].to('cpu'),"labels":i[1]["labels"].to('cpu')})

  print(f"step {j} from {len(test_dataset)}")
  j+=1

In [13]:
map.update(x,y)
map.compute()

{'map': tensor(0.4356),
 'map_50': tensor(0.7035),
 'map_75': tensor(0.5144),
 'map_large': tensor(0.6143),
 'map_medium': tensor(0.5010),
 'map_per_class': tensor(-1.),
 'map_small': tensor(0.0065),
 'mar_1': tensor(0.4304),
 'mar_10': tensor(0.4707),
 'mar_100': tensor(0.4707),
 'mar_100_per_class': tensor(-1.),
 'mar_large': tensor(0.7000),
 'mar_medium': tensor(0.5439),
 'mar_small': tensor(0.0222)}